## How to Connect to LLM API Endpoints

Here will cover:
- What are SDKs?
- Gemini, Openai SDKs 
- cURL & Python requests alternative

Note: With calling openai we are using the responses api endpt.

First let's load the api keys from env.

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Easiest way to access is through an SDK

SDK = Software Development Kit.
It’s a library (plus docs + helpers) that a company provides so you don’t have to write low-level API calls yourself.

Instead of manually crafting curl or requests.post(...), the SDK gives you functions and classes in your programming language.

For LLMs specifically the SDK is just a wrapper around the raw REST API of the model provider (OpenAI, Google Gemini, Anthropic, etc.).

It usually does things like:
- Handle authentication for you (you just set API_KEY once).
- Simplify sending prompts (model.generate_content("hello") instead of JSON bodies).
- Parse the responses into friendly objects (instead of raw JSON).

### Connect to Gemini via Gemini SDK

In [18]:
import google.generativeai as genai

# Configure Gemini with your API key
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create a model with system instructions and generation config
model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    system_instruction="You are a helpful assistant that always answers politely and succintly.",
    generation_config={
        "max_output_tokens": 30,   # cap response length (~words×4 = tokens)
        "temperature": 0.7,        # creativity
        "top_p": 0.9,              # nucleus sampling
        "top_k": 40                # top-k sampling
    }
)

# Generate a response with the user prompt
resp = model.generate_content("What do cats like to eat?")
print(resp.text)


Cats enjoy a variety of foods, including meat, fish, and specially formulated cat food.



### Connect to OpenAI via OpenAI SDK

In [19]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

resp = client.chat.completions.create(
    model="gpt-4o-mini",   # or "gpt-4o", "gpt-3.5-turbo", etc.
    messages=[
        {"role": "system", "content": "You are a polite assistant who always answers concisely."},
        {"role": "user", "content": "Say hi in 5 words."}
    ],
    max_tokens=30,       # cap response length
    temperature=0.7,     # creativity
    top_p=0.9,           # nucleus sampling
    frequency_penalty=0, # discourage repetition
    presence_penalty=0   # encourage new topics
)

print(resp.choices[0].message.content)


Hello! How are you today?


Interesting is that we can also call Gemini through the OpenAI SDK. Google exposed a compatibility layer that lets you call Gemini models using the OpenAI SDK style (the responses.create API).
- Endpoint: https://generativelanguage.googleapis.com/v1beta/openai/
- Auth: still your Gemini API key, not an OpenAI key.

Google did this because it helps people who are on the OpenAI SDK to quickly try Gemini's model without rewriting much code. It's not perfect parity - Gemini's advanced features are only native in the Gemini SDK or REST API.

In [20]:
from openai import OpenAI

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],  # still a Gemini key
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

resp = client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=[{"role": "user", "content": "What is a dog?"}]
)

print(resp.choices[0].message.content)  # -> "Hello there, how are you?"

A dog, scientifically known as *Canis lupus familiaris*, is a domesticated mammal belonging to the Canidae family, which also includes wolves, foxes, and jackals.

Here's a breakdown of what that means and some key characteristics of dogs:

*   **Domesticated:** Dogs are descendants of wolves and were domesticated by humans thousands of years ago. Domestication means they have been selectively bred and adapted to live alongside humans, exhibiting behavioral traits and physical characteristics different from their wild ancestors.
*   **Mammal:** They are warm-blooded animals with fur or hair, give birth to live young, and nourish their offspring with milk produced by mammary glands.
*   **Canidae:** This is the biological family that includes all dog-like carnivores.
*   **Carnivore (but often omnivorous):** While classified as carnivores, dogs have evolved to digest a wider range of foods, including grains and vegetables, often making them omnivores in practice.

**Key Characteristics:

## Send raw HTTP requests using cURL or Python requests

The core idea is that:
- cURL = a command-line tool to send HTTP requests
- requests = a python library for making same HTTP requests inside code.

Both:
- Send HTTP requests (GET, POST, etc.).
- Add headers (like Authorization).
- Send JSON bodies.
- Get back JSON you can read.

### cURL using Gemini

In [21]:
%%bash
# load .env so GEMINI_API_KEY is available in this subshell
export $(grep -v '^#' .env | xargs)

# Call Gemini API
curl \
  -H "Content-Type: application/json" \
  -H "x-goog-api-key: $GEMINI_API_KEY" \
  -d '{
    "contents": [{
      "role": "user",
      "parts": [{"text":"Explain to me what a dog is. Be succint"}]
    }],
    "generationConfig": {
      "maxOutputTokens": 20,
      "temperature": 0.7,
      "topP": 0.9,
      "topK": 40
    }
  }' \
  https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1000    0   755  100   245    874    283 --:--:-- --:--:-- --:--:--  1158  100   245    874    283 --:--:-- --:--:-- --:--:--  1158

{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "A dog is a domesticated carnivorous mammal, descended from wolves, known for its loyalty, trainability,"
          }
        ],
        "role": "model"
      },
      "finishReason": "MAX_TOKENS",
      "avgLogprobs": -0.12728489012945265
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 11,
    "candidatesTokenCount": 21,
    "totalTokenCount": 32,
    "promptTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 11
      }
    ],
    "candidatesTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 21
      }
    ]
  },
  "modelVersion": "gemini-2.0-flash",
  "responseId": "FCmjaN2iFZ2gz7IP49js2QY"
}


### cURL using OpenAI

In [22]:
%%bash

curl https://api.openai.com/v1/responses \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -H "Content-Type: application/json" \
  -d '{"model":"gpt-4.1-mini","input":"Say hi in 5 words."}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1444  100  1391  100    53   1883     71 --:--:-- --:--:-- --:--:--  1959  100    53   1883     71 --:--:-- --:--:-- --:--:--  1959

{
  "id": "resp_68a3291533cc81a1bf76754e284a065405ca8cbe41284a83",
  "object": "response",
  "created_at": 1755523349,
  "status": "completed",
  "background": false,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "max_output_tokens": null,
  "max_tool_calls": null,
  "model": "gpt-4.1-mini-2025-04-14",
  "output": [
    {
      "id": "msg_68a32915761c81a1b1d3ac0faa8c347205ca8cbe41284a83",
      "type": "message",
      "status": "completed",
      "content": [
        {
          "type": "output_text",
          "annotations": [],
          "logprobs": [],
          "text": "Hello! Hope you're doing great!"
        }
      ],
      "role": "assistant"
    }
  ],
  "parallel_tool_calls": true,
  "previous_response_id": null,
  "prompt_cache_key": null,
  "reasoning": {
    "effort": null,
    "summary": null
  },
  "safety_identifier": null,
  "service_tier": "default",
  "store": true,
  "temperature": 1.0,
  "text": {
    "format": {
      "type": "text"
   

### Python requests using Gemini

In [23]:
import requests, json

url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

headers = {
    "Content-Type": "application/json",
    "x-goog-api-key": GEMINI_API_KEY,   
}
data = {
    "contents": [{
        "role": "user",
        "parts": [{"text": "Explain to me what a dog is. Be succinct"}]
    }],
    "generationConfig": {
        "maxOutputTokens": 20,
        "temperature": 0.7,
        "topP": 0.9,
        "topK": 40
    }
}

resp = requests.post(url, headers=headers, json=data)

# Pretty-print the whole response JSON
print(json.dumps(resp.json(), indent=2))

# Or extract just the text
print(resp.json()["candidates"][0]["content"]["parts"][0]["text"])


{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "A dog is a domesticated mammal, descended from wolves, known for its loyalty, trainability, and"
          }
        ],
        "role": "model"
      },
      "finishReason": "MAX_TOKENS",
      "avgLogprobs": -0.16972227096557618
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 10,
    "candidatesTokenCount": 20,
    "totalTokenCount": 30,
    "promptTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 10
      }
    ],
    "candidatesTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 20
      }
    ]
  },
  "modelVersion": "gemini-2.0-flash",
  "responseId": "FimjaI3vNbvSz7IPx_mvwAY"
}
A dog is a domesticated mammal, descended from wolves, known for its loyalty, trainability, and


### Python requests using OpenAI

In [24]:
import requests

url = "https://api.openai.com/v1/responses"
headers = {
    "Authorization": f"Bearer {OPENAI_API_KEY}",
    "Content-Type": "application/json",
}
data = {
    "model": "gpt-4.1-mini",
    "input": "Say hi in 5 words."
}
r = requests.post(url, headers=headers, json=data)
print(r.json()["output"][0]["content"][0]["text"])


Hello! Hope you're doing well!
